In [1]:
import tensorflow as tf
import os
import numpy as np

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [9]:
tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))))
decode_tokenize_text =  tf.keras.layers.StringLookup(mask_token=None, vocabulary=list(sorted(set(text))), invert=True)
tokenize_text(['a'])

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([40], dtype=int64)>

In [17]:
def backward_text(arr):
    return tf.strings.reduce_join(decode_tokenize_text(arr), axis=-1).numpy()

In [19]:
all_tokens = tokenize_text(tf.strings.unicode_split(text, 'UTF-8'))

tensor_set = tf.data.Dataset.from_tensor_slices(all_tokens)

In [36]:
sequence_length = 100
epoch_exemple_length = len(text) // (sequence_length + 1)

# O método batch permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.

# We're gonna predict the next character using the RNN, so, to predict the word Hello, if the input is Hell, the output will be ello
dataset = tensor_set.batch(batch_size=sequence_length+1, drop_remainder=True).map(lambda sequence: [sequence[:-1], sequence[1:]])

batch_size = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
buffer_size = 10000

dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE))

vocab_size = len(tokenize_text.get_vocabulary())

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size,
    embedding_dim=256,
    rnn_units=1024)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)

model.fit(dataset, epochs=100, callbacks=[checkpoint_callback, early_stop])

Epoch 1/100


 11/172 [>.............................] - ETA: 10:43 - loss: 4.1790

: 

: 